In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Codey API 入門 - 程式碼完成功能

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/code/code_completion.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/code/code_completion.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/code/code_completion.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Lavi Nigam](https://github.com/lavinigam-gcp), [Polong Lin](https://github.com/polong-lin) |


## 概要

此筆記本旨在提供一個使用 [Codey 模型](https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview)(特別是 `code-gecko` 模型) 進行程式碼完成的實作入門。你將會學習如何建立提示，與 `code-gecko` 模型互動，並根據你編寫的程式碼脈絡產生程式碼建議。


### Vertex AI PaLM API
Vertex AI PaLM API，[於 2023 年 5 月 10 日發布](https://cloud.google.com/vertex-ai/docs/generative-ai/release-notes#may_10_2023)，由 [PaLM 2](https://ai.google/discover/palm2) 支援。

### 使用 Vertex AI PaLM API

你可以使用以下方法與 Vertex AI PaLM API 互動：

* 使用 [Generative AI Studio](https://cloud.google.com/generative-ai-studio) 進行快速測試和指令產生。
* 在 Cloud Shell 中使用 cURL 指令。
* 在 Jupyter 筆記本中使用 Python SDK

本筆記本著重於使用 Python SDK 來呼叫 Vertex AI PaLM API。如要取得關於不撰寫程式碼即可使用 Generative AI Studio 的更多資訊，可以探索 [UI 指令入門](https://github.com/doggy8088/generative-ai/blob/main/language/intro_vertex_ai_studio.md)。


如要取得更多資訊，請查看 [Vertex AI 生成式 AI 支援文件](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)。


### 目標

在這個教學課程中，你將學習各種程式碼完成範例，以完成：
* 函式
* 類別
* 語句
* 表達式及變數
* 匯入


### 費用
本教學指南使用 Google
Cloud 的計費元件：

* Vertex AI Generative AI Studio

了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)
，並使用 [定價計算器](https://cloud.google.com/products/calculator/)
，根據預計用量產生成本估算。


## 開始使用


### 安裝 Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**僅限 Colab：** 取消以下Cell註解以重新啟動核心或使用按鈕重新啟動核心。對於 Vertex AI Workbench，你可以使用右上方的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 驗證筆記本環境
* 如果你使用 **Colab** 執行此筆記本，取消註解下方的Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看[此處](https://github.com/doggy8088/generative-ai/tree/main/setup-env)的設定說明。


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

## 程式碼補完


什麼是程式碼補完功能？

程式碼補完功能是許多整合開發環境 (IDE) 的功能，可在程式員輸入時，建議輸入的程式碼。這可以節省時間，並有助於避免錯誤。程式碼補完建議會依據正在編寫的程式碼的內容而定，例如程式語言、目前的程式碼列，以及已定義的變數。



使用程式碼補完功能的好處？

一般而言，使用程式碼補完功能有好幾項好處，包括：

* **提升生產力：** 程式碼補完功能可以在程式員輸入時提供程式碼建議，這樣可以為程式員省下許多時間。這可以讓他們騰出時間專注執行其他任務，例如設計軟體的架構或偵錯程式碼。

* **減少錯誤：** 程式碼補完功能可以建議語法正確且語意有意義的程式碼，有助於減少錯誤。當程式員使用陌生的或不熟悉的程式語言或 API 時，它會特別有幫助。

* **改善程式碼品質：** 程式碼補完功能可以建議與專案的樣式指南相符的程式碼，有助於改善程式碼品質。這可以讓程式碼更易於閱讀和維護。


程式碼補完和 IDE 整合：

當透過 Codey Model 的程式碼補完功能與 IDE 整合時，它可能會更加強大。IDE 可以利用其對專案架構和程式碼庫的瞭解，提供更準確且相關的建議。例如，如果程式員在類別中輸入程式碼，IDE 可以建議從該類別傳送的方法和欄位。

以下是使用與不同 IDE 整合的程式碼補完功能的好處：

* **提升生產力：** 程式碼補完功能可以幫助程式員更快速、更準確地撰寫程式碼，這可能帶來顯著的生產力提升。
* **改善程式碼品質：** 程式碼補完功能可以幫助程式員避免錯誤和輸入錯誤，它也可以建議更高效率且慣用的程式碼。
* **提升程式碼可讀性：** 程式碼補完功能可以建議一致的變數名稱和函式簽章，有助於程式員撰寫出可讀性和可維護性較高的程式碼。
* **縮短學習曲線：** 程式碼補完功能可以建議程式員使用正確的符號和函式，有助於新手程式員更快地學習新的語言和架構。


### 匯入函式庫


**僅限 Colab：** 取消下方單元格的註解，以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此動作。


In [ ]:
# import vertexai

# PROJECT_ID = ""  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from IPython.display import Markdown, display
from vertexai.language_models import CodeGenerationModel

## 程式碼補完 with code-gecko@002


Vertex AI Codey API 包括程式碼完成 API，支援基於最近編寫的程式碼提供程式碼建議。使用命名為 `code-gecko` 的生成式 AI 基礎模型與程式碼完成 API 互動。

若要深入瞭解如何為程式碼完成建立提示，請參閱 [建立程式碼完成提示](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/code-completion#:~:text=code%20completion%2C%20see-,Create%20prompts%20for%20code%20completion,-.).

程式碼完成 API 的參數比程式碼生成多一些。

* prefix：必要：對於程式碼模型，prefix 代表有意義的程式碼片段開頭，或用於描述要產生的程式碼的自然語言提示。

* suffix：可選：對於程式碼完成，suffix 代表有意義的程式碼片段結尾。模型會嘗試填補 prefix 與 suffix 之間的程式碼。

* temperature：必要：溫度控制 Token 選擇的隨機程度。與其他模型相同。範圍：(0.0 - 1.0，預設 0)

* maxOutputTokens：必要：回應中可以產生的最大 Token 數。範圍：(1 - 64，預設 64)

* stopSequences：可選：指定字串清單，告知模型如果在回應中遇到其中一個字串，就停止產生文字。這些字串區分大小寫。


### 載入模型


In [ ]:
code_completion_model = CodeGenerationModel.from_pretrained("code-gecko@latest")

### Hello Codey Completion


#### Python


In [ ]:
prefix = """def find_x_in_string(string_s, x):

         """

response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """
         def reverse_string(s):
            return s[::-1]
         def test_empty_input_string():
         """

response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

#### Java


In [ ]:
prefix = """
        ArrayList<String> myList = new ArrayList<>();
        //add the `String` "Hello, world!" to the `ArrayList`:
         """

response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """
        public static List<String> getUniqueStrings(List<String> strings) {
         """

response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """
        String[] names = {"Alice", "Bob", "Carol"};
        for (String name : names) {
         """

response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

#### JavaScript


In [ ]:
prefix = """
        #javaScript
        function factorial(n) {
         """

response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """
        function greet(name) {
            return "Hello, " + name + "!";
          }
        const greeting = greet(YOUR_NAME_HERE);
         """

response = code_completion_model.predict(prefix=prefix)

print(response.text)

#### C/C++


In [ ]:
prefix = """
        int main() {
          char str[] = "Hello, world!";
         """

response = code_completion_model.predict(prefix=prefix)

print(response.text)

In [ ]:
prefix = """
        LinkedList linkedList;

        linkedList.addNode(1);
        linkedList.addNode(2);
        linkedList.addNode(3);

        int value =
         """

response = code_completion_model.predict(prefix=prefix)

print(response.text)

### 碼完成範例：


#### 完成函式


In [ ]:
prefix = """import math
            # Start typing the name of a function
            def sqrt(x):
         """

response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """def greet(name):
              print(f"Hello, {name}!")

            # Call the greet() function
         """

response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

#### 完課


In [ ]:
prefix = """class Dog:
              def bark(self):
                print("Woof!")

            # Create a new Dog object
          """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """class Person:
              #Represents a person.
              def __init__(self, name, age):
                self.name = name
                self.age = age

            # Start typing the name of the Person class
            Person(
          """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

#### 完成功能敘述


In [ ]:
prefix = """if age >= 21:
              print("You are an adult. ")
         """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """if x < 10:
              # Complete the statement
         """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

#### 補充表達式


In [ ]:
prefix = """x = 10 +
         """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """1 + 2 * 3
         """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

#### 完成變數


In [ ]:
prefix = """# Define a variable
            name = "Alice"
            #get uppercase of the variable
            name.upper()
         """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """x = 10
            y = x +
         """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

#### 完成匯入


In [ ]:
prefix = """import math
            import numpy as np
            #import machine learning libraries
         """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

In [ ]:
prefix = """import math
            import time
            import random
            import sys
         """
response = code_completion_model.predict(prefix=prefix, max_output_tokens=64)

print(response.text)

### 迴饋循環程式碼補完


In [ ]:
prefix = "def find_max_element(list):"
i = 0
while i < 3:
    response = code_completion_model.predict(
        prefix=prefix,
    )
    print(response.text)
    prefix = response.text
    i += 1

In [ ]:
prefix = """class Dog:
              def bark(self):
                print("Woof!")
          """
i = 0
while i < 3:
    response = code_completion_model.predict(
        prefix=prefix,
    )
    print(response.text)
    prefix = response.text
    i += 1

### 最佳實務


#### **如何撰寫有效的程式碼補全提示** 

在撰寫程式碼補全提示時，具體並描述得越詳細越好。提供給模型更多的資訊，它就能越了解你要達成的目標。

以下是撰寫有效程式碼補全提示的一些秘訣：

* 以自然語言說明模型要產生的內容作為開頭。這必須是你目標的明確且簡潔敘述，例如「完成下列函式以列印兩個數字的總和」或「產生一個函式來檢查字串是否為迴文」。

* 在提示中包含任何相關語境。這可以包含你已經撰寫的程式碼、你使用的程式語言或模型可能需要知道的其他資訊。

* 使用範例來說明你要找什麼。如果你可以，提供模型所需產生的程式碼範例。這將有助於模型更了解你的意圖。
以下是一個良好的程式碼補全提示範例：




```
完成下列 Python 函式以檢查字串是否為迴文：

def is_palindrome(string):
  """檢查字串是否為迴文。

  Args：
    字串：字串。

  傳回：
    布林值，表示該字串是否為迴文。
  """
  # TODO：實作此函式。
```


#### **如何選擇合適的溫度和最大輸出 Token** 

溫度和最大輸出 Token 是控制程式碼完成模型行為的兩個重要參數。

* 溫度：溫度控制著模型的創造力。較高的溫度將導致更多有創意和出乎意料的建議，而較低的溫度會導致更加保守和可預測的建議。

* 最大輸出 Token：最大輸出 Token 控制著模型可以生成的程式碼的最大長度。較高的最大輸出 Token 將允許模型產生更長的程式碼片段，而較低的輸出 Token 將限制所產生程式碼的長度。

在選擇正確的溫度和最大輸出 Token 時，考慮你嘗試完成的具體任務非常重要。如果你需要該模型產生有創意和出乎意料的建議，則應該使用較高的溫度。如果你需要該模型產生特定長度的程式碼片段，則應該使用適當的最大輸出 Token。


**#### 如何解讀和使用程式碼完成建議** 

一旦你生成了某些程式碼完成建議，仔細解讀並使用它們十分重要。

程式碼完成模型並非完美，有可能會產生不正確或不完備的建議。仔細檢閱建議並在程式碼中使用前進行測試十分重要。

以下是解讀和使用程式碼完成建議的一些秘訣：

* 確保建議句法正確。程式碼完成模型可能會產生句法不正確的建議。在程式碼中使用它們之前，檢查建議的句法十分重要。

* 在使用建議於程式碼前測試它們。一旦你找到一些令你滿意的建議，在你的程式碼裡使用它們之前測試它們十分重要。這有助於確保建議正確無誤且能按預期運作。


#### **避免常見程式碼完成功能陷阱的方法** 

以下列出一些常見的程式碼完成功能陷阱，你應避免陷入：

* 不要依賴於程式碼完成功能模型來生成你的所有程式碼。程式碼完成功能模型是一種工具，但它不應被用來生成你的所有程式碼。了解你正在編寫的程式碼，並能夠仔細地檢閱和測試它，這一點很重要。

* 不要在不理解程式碼完成功能建議的情況下使用它們。在你的程式碼中使用程式碼完成功能建議之前，了解這些建議很重要。這將有助於你找出任何潛在的錯誤或問題。

* 不要將程式碼完成功能建議用於太複雜的任務。程式碼完成功能模型不是用來生成複雜的程式碼片段的。如果你需要生成複雜的程式碼，最好自己編寫。
